# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

weather_df = pd.read_csv("../WeatherPy/CityWeatherData.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kruisfontein,-34.0033,24.7314,58.44,71,6,6.91,ZA,1651800879
1,brigham city,41.5102,-112.0155,72.86,26,0,7.00,US,1651800879
2,mataura,-46.1927,168.8643,64.24,61,100,2.30,NZ,1651800880
3,avarua,-21.2078,-159.7750,82.45,69,20,6.91,CK,1651800880
4,kendari,-3.9450,122.4989,86.27,68,16,1.66,ID,1651800881


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key = g_key)
location = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [6]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map

fig_humidity = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                point_radius = 4)
fig_humidity.add_layer(heat_layer)
fig_humidity


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Create dataframe and empty column

hotel_df = weather_df.loc[(weather_df["Max Temp"] <90) & (weather_df["Max Temp"] >76)
                       & (weather_df["Humidity"] <70) & (weather_df["Cloudiness"] == 1), :]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

hotel_df_1 = pd.DataFrame(hotel_df)
hotel_df_1["Hotel Name"] = " "
hotel_lat = hotel_df_1["Lat"]
hotel_lng = hotel_df_1["Lng"]
hotel_df_1

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,dalbandin,28.8947,64.4101,80.62,14,1,4.74,PK,1651800936,
1,mengcheng,33.2579,116.5655,80.91,39,1,7.02,CN,1651800962,
2,linhai,28.8500,121.1167,80.92,44,1,2.15,CN,1651801017,


In [19]:
# Loop through dataframe and add nearby hotel result
hotel_name = []

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_result = requests.get(base_url, params=params).json()
    
    try:
#         hotel_name.append(hotel_result["results"][0]["name"])
        hotel_df_1.loc[index, "Hotel Name"] = hotel_result["results"][0]["name"]
    
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
    
hotel_df_1


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,dalbandin,28.8947,64.4101,80.62,14,1,4.74,PK,1651800936,Iqbal Hotel
1,mengcheng,33.2579,116.5655,80.91,39,1,7.02,CN,1651800962,Jiayue Hotel
2,linhai,28.8500,121.1167,80.92,44,1,2.15,CN,1651801017,Jiutian Commerce Hotel


In [20]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map



# Display figure

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                dissipating=False, max_intensity=74,
                                point_radius = 4)


marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))